In [1]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import math
import torch
from torch import nn
from torch import optim
from tensorboardX import SummaryWriter
from datetime import datetime
import time
import pygame
import load
from models import DynamicTopologyModel
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device="cpu"
EXPERIMENT=11
writer = SummaryWriter('runs/experiment_{}/'.format(EXPERIMENT))

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
cells =[(6, 0), (25,0)]
loads = [(0, 0)]
load.calculate_cell_load(cells, loads)

[(6, 0, 0.6317491006820891), (25, 0, 0.3682508993179109)]

In [3]:
0.2 * 32

6.4

In [4]:
def save_parameters(writer, model, batch_idx):
    for k, v in model.state_dict().items():
        shape = v.shape
        # Don't do this for single weights or biases
        if np.any(np.array(shape) > 1):
            mean = torch.mean(v)
            std_dev = torch.std(v)
            maximum = torch.max(v)
            minimum = torch.min(v)
            writer.add_scalars("{}_{} ".format(k, shape), {"mean": mean,
                                                                    "std_dev": std_dev,
                                                                    "max": maximum,
                                                                    "min": minimum}, batch_idx)
            writer.add_histogram("{}_{}".format(k, shape), v, batch_idx)
            
        else:
            writer.add_scalar("{}_{}".format(k, shape), v.data, batch_idx)

In [5]:
def test_parameters_update(last_parameters_np ,parameters_np):
    for last_param, param in zip(last_parameters_np, parameters_np):
        diff = param - last_param
        #print(last_parameters.max())
        print(diff.max())
        #print(not np.all(diff))


In [6]:
def print_example(reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
                  reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
                  neighbourhood_cell_present_target):
    print("reference_cell_input") 
    print(reference_cell_input)
    print("reference_cell_present_input") 
    print(reference_cell_present_input)
    print("neighbourhood_cell_rel_input") 
    print(neighbourhood_cell_rel_input)
    print("neighbourhood_cell_load_input") 
    print(neighbourhood_cell_load_input)
    print("neighbourhood_cell_present_input") 
    print(neighbourhood_cell_present_input)
    print("reference_cell_target") 
    print(reference_cell_target)
    print("reference_cell_present_target") 
    print(reference_cell_present_target)
    print("neighbourhood_cell_rel_target") 
    print(neighbourhood_cell_rel_target)
    print("neighbourhood_cell_present_target") 
    print(neighbourhood_cell_present_target)

In [7]:
BATCH_SIZE = 32
INPUT_SEQ_LEN = 3
TARGET_SEQ_LEN = 3
LEARNING_RATE = 1e-3
SAVE_VARIABLE_EVERY = 100
DATASET_WORKERS = 4


input_seq_len= torch.Tensor([INPUT_SEQ_LEN]).int()
target_seq_len= torch.Tensor([TARGET_SEQ_LEN]).int()

# Define Data.
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 8), initial_load_counts=(1,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.2, 0.2
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=DATASET_WORKERS)

# Define Model
if True:
    dynamic_topology_model = torch.load("model.pt")
else:
    dynamic_topology_model = DynamicTopologyModel(neighbourhood_hidden_size=64, neighbourhood_cell_count=6,
                     neighbourhood_output_size=16, lstm_hidden_size=64, lstm_layers=2, teacher_forcing_probability=0.5, device=device).to(device)

parameters = dynamic_topology_model.parameters

# Define optimizer
optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
last_parameters_np = list(np.copy(param.cpu().detach().numpy()) for param in parameters)

# Train
start = datetime.now()
for batch_idx, data in enumerate(dataloader):
    optimizer.zero_grad()
    reference_cell_input =  data[0].to(device)
    reference_cell_present_input = data[1].to(device)
    neighbourhood_cell_rel_input = data[2].to(device)
    neighbourhood_cell_load_input = data[3].to(device)
    neighbourhood_cell_present_input = data[4].to(device)
    reference_cell_target = data[5].to(device)
    reference_cell_present_target = data[6].to(device)
    neighbourhood_cell_rel_target = data[7].to(device)
    neighbourhood_cell_present_target = data[8].to(device)
    
    
    decoder_output_seq = dynamic_topology_model.forward(input_seq_len, target_seq_len, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
    
    loss = criterion(decoder_output_seq, reference_cell_target)
    loss.backward()
    optimizer.step()
    if batch_idx == 0:
        parameters = dynamic_topology_model.parameters
        parameters_np = list(param.cpu().detach().numpy() for param in parameters)
        test_parameters_update(last_parameters_np, parameters_np)
        inputs =(input_seq_len, target_seq_len, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
        writer.add_graph(dynamic_topology_model, inputs, True)
    if batch_idx % SAVE_VARIABLE_EVERY == 0:
        torch.save(dynamic_topology_model, "model.pt")
        #print_example(reference_cell_input, reference_cell_present_input,
        #            neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
        #              reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
        #              neighbourhood_cell_present_target)    
        end = datetime.now()
        print("{} Examples/sec".format((SAVE_VARIABLE_EVERY* BATCH_SIZE) / ((end - start).total_seconds()) ))
        start = datetime.now()
        print("loss: {}".format(loss))
        #print("output: {}".format(decoder_output_seq))
        #print("target: {}".format(reference_cell_target))
        writer.add_scalar("loss", loss, batch_idx)
        save_parameters(writer, dynamic_topology_model, batch_idx)


0.009859383
0.009928405
0.009967744
0.009987831
0.00916107
-0.009991638
0.009883206
-0.009998828
0.009999394
0.0
0.009999145
0.009999145
0.0099981725
0.0
0.009999733
0.00999973
0.0
0.0
0.009999037
0.009999692
0.009999931
0.009999931
0.009999827
0.009997591
0.009999707
0.009999707
0.009999678
0.009998847
0.009999925
0.009999925
0.009999968
0.009999996


/Users/nsgoodger/personal/dynamic_topology/models.py:196: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  (batch_size, target_seq_len.data.numpy()[0], 1), device=self.device
/Users/nsgoodger/personal/dynamic_topology/models.py:200: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  for input_seq_idx in range(input_seq_len):
/Users/nsgoodger/personal/dynamic_topology/models.py:231: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in

graph(%0 : Int(1)
      %1 : Int(1)
      %2 : Float(32, 3, 1)
      %3 : Float(32, 3, 1)
      %4 : Float(32, 3, 8, 2)
      %5 : Float(32, 3, 8, 1)
      %6 : Float(32, 3, 1)
      %7 : Float(32, 3, 1)
      %8 : Float(32, 3, 8, 2)
      %9 : Float(64, 2)
      %10 : Float(64)
      %11 : Float(64, 1)
      %12 : Float(64)
      %13 : Float(1, 64)
      %14 : Float(1)
      %15 : Float(1, 64)
      %16 : Float(1)
      %17 : Float(256, 3)
      %18 : Float(256, 64)
      %19 : Float(256)
      %20 : Float(256)
      %21 : Float(256, 64)
      %22 : Float(256, 64)
      %23 : Float(256)
      %24 : Float(256)
      %25 : Float(1, 64)
      %26 : Float(1)
      %27 : Float(64, 2)
      %28 : Float(64)
      %29 : Float(16, 64)
      %30 : Float(16)
      %31 : Float(256, 18)
      %32 : Float(256, 64)
      %33 : Float(256)
      %34 : Float(256)
      %35 : Float(256, 64)
      %36 : Float(256, 64)
      %37 : Float(256)
      %38 : Float(256)
      %39 : Float(1, 64)
      %40 : Floa

KeyboardInterrupt: 

In [38]:
def add_cell_if_not_exists(new_cell, cells):
    cell_already_exists = False

    for i, cell in enumerate(cells):
        if cell[0] == new_cell[0] and cell[1] == new_cell[1]:
            cell_already_exists = True
            break
    if not cell_already_exists:
        cells.append(new_cell)
    return cells

def remove_cell_if_exists(remove_cell, cells):
    for i, cell in enumerate(cells):
        if cell[0] == remove_cell[0] and cell[1] == remove_cell[1]:
            cells.pop(i)
            break
    return cells


In [40]:
PIXELS = 640
LEFT = 1
RIGHT = 3
INPUT_SCREEN_COUNT = 3
TARGET_SCREEN_COUNT = 3
SCREEN_COUNT = INPUT_SCREEN_COUNT + TARGET_SCREEN_COUNT

PIXELS_PER_BLOCK = int(PIXELS / IMAGE_SIZE)
SCALE_ARR = np.ones((PIXELS_PER_BLOCK, PIXELS_PER_BLOCK, 1))

load_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 1))
#load_screen[1,12, 0] = 255
#load_screen[22,22, 0] = 255
loads = [(1, 12), (22, 22)]


screen = pygame.display.set_mode((PIXELS, PIXELS))
screens = []

screen_draw_idx = 0
last_pos = (0, 0)
color = (255, 128, 0)
radius = 10
block_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))
model_screen = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)
frame = 0
cells_seq = [list() for i in range(SCREEN_COUNT)]
#cells_load_seq = [list() for i in range(SCREEN_COUNT)]
BATCH_SIZE=1
show_labels = True
input_seq_len= torch.Tensor([INPUT_SCREEN_COUNT]).int()
target_seq_len= torch.Tensor([TARGET_SCREEN_COUNT]).int()

load_cells_seq_input_0, load_cells_seq_target_0, loads = load_topology_dataset.LoadCellDataset.generate_inputs_and_targets(INPUT_SCREEN_COUNT, TARGET_SCREEN_COUNT,
                                            (2, 2), (8,8), (0.2, 0.2))

cells_load_seq = load_cells_seq_input_0 + load_cells_seq_target_0
#print(cells_load_seq)
for seq_idx in range(len(cells_load_seq)):
    for cell_idx in range(len(cells_load_seq[seq_idx])):
        cells_seq[seq_idx].append((cells_load_seq[seq_idx][cell_idx][0], cells_load_seq[seq_idx][cell_idx][1]))

print(loads)
for load_item in loads:
    load_screen[load_item[0], load_item[1], 0] = 255

    
block_img = np.kron(load_screen, SCALE_ARR)    
while True:
    
    ##############################
    # Use model to forecast future load
    ##############################
    load_cells_seq_input = cells_load_seq[:INPUT_SCREEN_COUNT]
    load_cells_seq_target = cells_load_seq[INPUT_SCREEN_COUNT:]
    
 
    model_target = [list() for i in range(TARGET_SCREEN_COUNT)]
    if screen_draw_idx < INPUT_SCREEN_COUNT:
        caption = "Input frame {}".format(screen_draw_idx)
    elif show_labels:
        caption = "Label frame {}".format(screen_draw_idx)
    else:
        caption = "Model frame {}".format(screen_draw_idx)
    pygame.display.set_caption(caption)
    for seq_idx  in range(TARGET_SCREEN_COUNT):
        for reference_cell in cells_seq[INPUT_SCREEN_COUNT + seq_idx]:
            #print(reference_cell)
            ref_x, ref_y = reference_cell[0], reference_cell[1]

            # Get inputs from state.
            (reference_cell_input,
             reference_cell_present_input,
             neighbourhood_cell_rel_input,
             neighbourhood_cell_load_input,
             neighbourhood_cell_present_input,
             reference_cell_target,
             reference_cell_present_target,
             neighbourhood_cell_rel_target,
             neighbourhood_cell_present_target,) = load_topology_dataset.LoadCellDataset.build_inputs_and_targets(INPUT_SCREEN_COUNT,
                                                                                                 TARGET_SCREEN_COUNT,
                                                                                                 ref_x, ref_y,
                                                                                                 load_cells_seq_input,
                                                                                                 load_cells_seq_target)
            # Infer output from model.  Add batch dimension to each input since dataloader is not used here.
            decoder_output_seq = dynamic_topology_model.forward(input_seq_len, target_seq_len,
                                                                reference_cell_input.unsqueeze(0),
                                                                reference_cell_present_input.unsqueeze(0),
                                                                neighbourhood_cell_rel_input.unsqueeze(0),
                                                                neighbourhood_cell_load_input.unsqueeze(0),
                                                                reference_cell_target.unsqueeze(0),
                                                                reference_cell_present_target.unsqueeze(0),
                                                                neighbourhood_cell_rel_target.unsqueeze(0))

            model_target[seq_idx].append((ref_x, ref_y, decoder_output_seq.detach().numpy()[0][seq_idx][0]))    
    ################################
    # DISPLAY
    #################################
    screen_draw = pygame.Surface((PIXELS, PIXELS))
    
    #print(cells_seq)
    cells = cells_seq[screen_draw_idx]
    cell_loads = cells_load_seq[screen_draw_idx]

    if (screen_draw_idx < INPUT_SCREEN_COUNT) or (show_labels and screen_draw_idx >= INPUT_SCREEN_COUNT):
        for cell in cell_loads:
            color = (cell[2] * 1024) - 256
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,),
            )
    else:
        model_loads = model_target[screen_draw_idx - INPUT_SCREEN_COUNT]
        for cell in model_loads:
            #color = (cell[2] * 2024) - 512
            color = (cell[2] * 255)
            if color > 255:
                color = 255
            if color < 0:
                color = 0
            pygame.draw.rect(
            screen_draw,
            (color, 50, 0),
            (
                cell[0] * PIXELS_PER_BLOCK,
                cell[1] * PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,
                PIXELS_PER_BLOCK,),
            )

    e = pygame.event.poll()


    if e.type == pygame.QUIT:
        raise StopIteration
    if e.type == pygame.MOUSEBUTTONDOWN:
        leftclick, middleclick, rightclick = pygame.mouse.get_pressed()
        x, y = e.pos
        x_cell= int(x / PIXELS_PER_BLOCK)
        y_cell = int(y / PIXELS_PER_BLOCK)
        if leftclick and (len(cells) < load_topology_dataset.MAX_CELL_COUNT):
            cells = add_cell_if_not_exists((x_cell, y_cell), cells)
        elif rightclick and (len(cells) > load_topology_dataset.MIN_CELL_COUNT):
            cells = remove_cell_if_exists((x_cell, y_cell), cells)            
        # Update cells
        cells_seq[screen_draw_idx] = cells

        
        # Update all cell loads
        for i, cells in enumerate(cells_seq):
            cells_load_seq[i] = load.calculate_cell_load(cells, loads)
        print(cells_load_seq)   
    if e.type == pygame.KEYDOWN:
        if e.key == pygame.K_SPACE:
            if show_labels:
                show_labels = False
            else:
                show_labels = True
                print(model_target[-1])
                print(load_cells_seq_target[-1])
        if e.key ==  pygame.K_LEFT:
            if screen_draw_idx > 0:
                screen_draw_idx -= 1
        if e.key ==  pygame.K_RIGHT:
            if screen_draw_idx < SCREEN_COUNT - 1:
                screen_draw_idx += 1
    pygame.display.flip()
    target_screen = pygame.surfarray.array3d(screen_draw)
    display_img = block_img + target_screen
    new_surf = pygame.pixelcopy.make_surface(display_img.astype(np.uint8))
    screen.blit(new_surf, (0, 0))


[(0, 22), (31, 20)]
[[(6, 12, 0.24192487124025913), (18, 22, 0.2765105156276669), (3, 19, 0.24547360412493308), (26, 17, 0.24658389284041463), (28, 22, 0.2389310001643411), (4, 24, 0.24966660078090755), (26, 27, 0.24215040281237277), (24, 19, 0.2587591124091049)], [(6, 12, 0.2715356394329423), (18, 22, 0.31497749434114014), (3, 19, 0.274303193054482), (26, 17, 0.28403047430144573), (4, 24, 0.2793139448500204), (26, 27, 0.27875835575801267), (24, 19, 0.29708089826195694)], [(6, 12, 0.3335857243436501), (3, 19, 0.33743501372998447), (4, 24, 0.343478227504465), (26, 27, 0.33988271955130556), (24, 19, 0.3625101662485407), (31, 12, 0.28310814862205413)], [(6, 12, 0.3335857243436501), (3, 19, 0.33743501372998447), (4, 24, 0.343478227504465), (26, 27, 0.33988271955130556), (24, 19, 0.3625101662485407), (31, 12, 0.28310814862205413)], [(6, 12, 0.5076609230777217), (4, 24, 0.5235359215466655), (24, 19, 0.5456905987420085), (31, 12, 0.4231125566336041)], [(4, 24, 0.957918871233844), (24, 19, 1.0

KeyboardInterrupt: 

In [ ]:
loads

In [ ]:
for seq_idx  in range(TARGET_SCREEN_COUNT):
    print(seq_idx)
    model_target[seq_idx].append((ref_x, ref_y, decoder_output_seq.detach().numpy()[0][seq_idx][0]))

In [ ]:

cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,24),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.2, 0.2])



In [ ]:
decoder_output.detach().numpy()[seq_idx][0]

In [ ]:
len(cells_load_seq[0])

In [42]:
cells_seq = [(cell_load[0], cell_load[1]) for cell_load in cell_load for cell_load_seq in cells_load_seq]

SyntaxError: invalid syntax (<ipython-input-42-8e3eb257a1ca>, line 1)